In [1]:
using JSON

#load OD pair-route incidence
odPairRoute = readall("od_pair_route_incidence_Sioux_simp.json");
odPairRoute = JSON.parse(odPairRoute);

#load link-route incidence
linkRoute = readall("link_route_incidence_Sioux_simp.json");
linkRoute = JSON.parse(linkRoute);

#load OD pair labels
odPairLabel = readall("od_pair_label_dict_Sioux_simp_refined.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readall("od_pair_label_dict__Sioux_simp_refined.json");
odPairLabel_ = JSON.parse(odPairLabel_);

#load link labels
linkLabel = readall("link_label_dict_Sioux_simp.json");
linkLabel = JSON.parse(linkLabel);

linkLabel_ = readall("link_label_dict_Sioux_simp_.json");
linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readall("node_link_incidence_Sioux_simp.json");
nodeLink = JSON.parse(nodeLink);

In [2]:
include("load_network_uni-class.jl")

load_ta_network (generic function with 2 methods)

In [3]:
ta_data = load_ta_network("Sioux_simp");

In [4]:
capacity = ta_data.capacity;
free_flow_time = ta_data.free_flow_time;

In [5]:
############
#Read in the demand file
file = open("../data_traffic_assignment_uni-class/SiouxFalls_trips_simp.txt")
demands = Dict{(Int64,Int64), Float64}()
s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = int(split(line)[2])
    else
        pairs = split(line, ";")
        for pair in pairs
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = int(pair_vals[1]), float(pair_vals[2])
                demands[(s,t)] = demand 
            end
        end
    end
end                
close(file)

In [6]:
demands

Dict{(Int64,Int64),Float64} with 81 entries:
  (9,5) => 800.0
  (3,6) => 300.0
  (6,9) => 400.0
  (4,4) => 0.0
  (9,4) => 700.0
  (8,9) => 800.0
  (9,1) => 500.0
  (3,1) => 100.0
  (4,5) => 500.0
  (9,9) => 0.0
  (2,4) => 200.0
  (6,5) => 200.0
  (4,9) => 700.0
  (8,4) => 700.0
  (1,2) => 100.0
  (9,2) => 200.0
  (5,1) => 200.0
  (9,8) => 800.0
  (3,4) => 200.0
  (9,6) => 400.0
  (8,2) => 400.0
  (2,3) => 100.0
  (2,6) => 400.0
  (5,4) => 500.0
  (2,1) => 100.0
  ⋮     => ⋮

In [7]:
odPairLabel_

Dict{String,Any} with 72 entries:
  "32" => {4,9}
  "29" => {4,6}
  "1"  => {1,2}
  "54" => {7,6}
  "2"  => {1,3}
  "41" => {6,1}
  "65" => {9,1}
  "53" => {7,5}
  "51" => {7,3}
  "27" => {4,3}
  "42" => {6,2}
  "33" => {5,1}
  "28" => {4,5}
  "52" => {7,4}
  "50" => {7,2}
  "63" => {8,7}
  "26" => {4,2}
  "10" => {2,3}
  "24" => {3,9}
  "58" => {8,2}
  "25" => {4,1}
  "23" => {3,8}
  "49" => {7,1}
  "59" => {8,3}
  "5"  => {1,6}
  ⋮    => ⋮

In [8]:
demandsVec = zeros(length(odPairLabel_))

for i = 1:length(demandsVec)
    demandsVec[i] = demands[(odPairLabel_["$i"][1], odPairLabel_["$i"][2])]
end

In [9]:
demandsVec

72-element Array{Float64,1}:
  100.0
  100.0
  500.0
  200.0
  300.0
  500.0
  800.0
  500.0
  100.0
  100.0
  200.0
  100.0
  400.0
    ⋮  
  500.0
  800.0
 1000.0
  800.0
  500.0
  200.0
  100.0
  700.0
  800.0
  400.0
  600.0
  800.0

In [10]:
for key=keys(odPairRoute)
    if contains(key, "72-")
        println(key)
    end
end

72-232
72-233


In [11]:
using JuMP
using Gurobi

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numLinks = 20
numRoute = 236
numOD = 72

@defVar(m, linkFlow[1:numLinks])
@defVar(m, pathFlow[1:numRoute])

pathFlowSum = Dict()

for i=1:numOD
    pathFlowSum[i] = 0
    for j=1:numRoute
        if "$(i)-$(j)" in keys(odPairRoute)
            pathFlowSum[i] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowSum[i] == demandsVec[i])
end

pathFlowLinkSum = Dict()

for a=1:numLinks
    pathFlowLinkSum[a] = 0
    for j=1:numRoute
        if "$(a)-$(j)" in keys(linkRoute)
            pathFlowLinkSum[a] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowLinkSum[a] == linkFlow[a])
end

for j=1:numRoute
    @addConstraint(m, pathFlow[j] >= 0)
end

@defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:numLinks})

@setNLObjective(m, Min, f)

solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1235
Number of nonzeros in inequality constraint Jacobian.:      233
Number of nonzeros in Lagrangian Hessian.............:       20

Total number of variables............................:      253
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [12]:
getValue(linkFlow)

20-element Array{Float64,1}:
 1800.0
 1400.0
 1800.0
 3100.0
 1400.0
 2200.0
 2200.0
 4400.0
 4400.0
 3900.0
 2700.0
 3100.0
 3900.0
 5200.0
 3400.0
 5200.0
 3400.0
 1400.0
 2700.0
 1400.0

In [13]:
getObjectiveValue(m)

227728.75351504557

In [14]:
using JuMP
using Gurobi

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numNodes = 9  # number of nodes

@defVar(m, linkFlow[1:numLinks])

@defVar(m, odLinkFlow[keys(demands), 1:numLinks])
for (s,t) = keys(demands)
    for k = 1:numLinks
        @addConstraint(m, odLinkFlow[(s,t), k] >= 0)
    end
end

for k = 1:numLinks
    linkFlowPartial = sum([odLinkFlow[(s,t), k] for (s,t) = keys(demands)])
    @addConstraint(m, linkFlowPartial == linkFlow[k])
end

for l = 1:numNodes
    for (s,t) = keys(demands)
        if s != t
            odLinkFlowPartial = sum([nodeLink["$(l-1)-$(k-1)"] * odLinkFlow[(s,t), k] for k = 1:numLinks])
            if (l == s)
                @addConstraint(m, odLinkFlowPartial + demands[(s,t)] == 0)
            elseif (l == t)
                @addConstraint(m, odLinkFlowPartial - demands[(s,t)] == 0)
            else
                @addConstraint(m, odLinkFlowPartial == 0)
            end
        end
    end
end

@defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:numLinks})

@setNLObjective(m, Min, f)

solve(m)

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     4520
Number of nonzeros in inequality constraint Jacobian.:     1620
Number of nonzeros in Lagrangian Hessian.............:       20

Total number of variables............................:     1640
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      668
Total number of inequality constraints...............:     1620
        inequality constraints with only lower bounds:     1620
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

:Optimal

In [15]:
getValue(linkFlow)

20-element Array{Float64,1}:
 1800.0
 1400.0
 1800.0
 3100.0
 1400.0
 2200.0
 2200.0
 4400.0
 4400.0
 3900.0
 2700.0
 3100.0
 3900.0
 5200.0
 3400.0
 5200.0
 3400.0
 1400.0
 2700.0
 1400.0

In [16]:
getObjectiveValue(m)

227728.75346819183

In [ ]:
1800.0
 1400.0
 1800.0
 3100.0
 1400.0
 2200.0
 2200.0
 4400.0
 4400.0
 3900.0
 2700.0
 3100.0
 3900.0
 5200.0
 3400.0
 5200.0
 3400.0
 1400.0
 2700.0
 1400.0

227728.75351504557